In [1]:
import pandas as pd
import numpy as np
path = r"C:\Users\kpras\Desktop\ClientTATApplicable_1To31Dec25.xlsx"
df = pd.read_excel(path)
print(df.head())

   SrNo                       Client                  Product ApplicationNo  \
0     1  Tata Motors Finance Limited  Tata Motors_CV_SCV_FTU1  R33225949428   
1     2  Tata Motors Finance Limited  Tata Motors_CV_SCV_FTU2  R32925947664   
2     3  Tata Motors Finance Limited  Tata Motors_CV_SCV_FTU2  R33425949939   
3     4  Tata Motors Finance Limited  Tata Motors_CV_SCV_FTU2  R33225949163   
4     5  Tata Motors Finance Limited  Tata Motors_CV_SCV_FTU2  R33125948528   

  ApplicationStatus             Case In Time            Case Out Time  \
0           Sampled  2025-11-30 17:08:10.250  2025-12-01 09:45:09.130   
1           Sampled  2025-11-30 17:12:31.680  2025-12-01 09:31:10.603   
2           Sampled  2025-11-30 17:21:02.553  2025-12-01 09:41:12.077   
3           Sampled  2025-11-30 17:36:59.657  2025-12-01 09:49:10.117   
4           Sampled  2025-11-30 17:44:08.940  2025-12-01 09:51:40.123   

            Processed By  ClientDecidedTAT CalculatedTAT In HH:MM  ...  \
0  Auto Samp

In [2]:
df_new = df[['Product', 'ApplicationStatus']].copy()
df_new['cnt'] = 1

product_sampling = pd.pivot_table(
    df_new,
    index='Product',
    columns='ApplicationStatus',
    values='cnt',
    aggfunc='sum',
    fill_value=0,
    margins=True,              # adds Grand Total
    margins_name='Grand Total'
)

print(product_sampling)


ApplicationStatus          Sampled  Screened  Grand Total
Product                                                  
Tata Motors_CV_ICV_FTU1         67        51          118
Tata Motors_CV_ICV_FTU2         59        71          130
Tata Motors_CV_ICV_RTLC1         1         5            6
Tata Motors_CV_ICV_SRT1         44        47           91
Tata Motors_CV_ICV_SRT2         11        15           26
Tata Motors_CV_LCV_FTU1         59        65          124
Tata Motors_CV_LCV_FTU2         25        42           67
Tata Motors_CV_LCV_RTLC1         3         4            7
Tata Motors_CV_LCV_RTLC2        10         5           15
Tata Motors_CV_LCV_SRT1         30        38           68
Tata Motors_CV_LCV_SRT2          6        14           20
Tata Motors_CV_LCV_SRT3          3         1            4
Tata Motors_CV_MHCV_FTU1       224       162          386
Tata Motors_CV_MHCV_FTU2        92       135          227
Tata Motors_CV_MHCV_RTLC1       22        34           56
Tata Motors_CV

In [3]:
# df_new = df[['Product', 'Application Status']]

# product_sampling = (
#     df_new.groupby(['Product', 'Application Status'])
#           .size()
#           .unstack(fill_value=0)
# )

# product_sampling['Total'] = product_sampling.sum(axis=1)
# product_sampling.loc['Total'] = product_sampling.sum()


# print(product_sampling)


In [4]:
# keep pivot with margins
product_sampling2 = pd.pivot_table(
    df_new,
    index='Product',
    columns='ApplicationStatus',
    values='cnt',
    aggfunc='sum',
    fill_value=0,
    margins=True,
    margins_name='Grand Total'
)

# calculate percentage WITHOUT Grand Total
product_sampling_pct = product_sampling2.div(
    product_sampling.drop(columns='Grand Total').sum(axis=1),
    axis=0
).mul(100).round(2)

print(product_sampling_pct)



ApplicationStatus          Sampled  Screened  Grand Total
Product                                                  
Tata Motors_CV_ICV_FTU1      56.78     43.22        100.0
Tata Motors_CV_ICV_FTU2      45.38     54.62        100.0
Tata Motors_CV_ICV_RTLC1     16.67     83.33        100.0
Tata Motors_CV_ICV_SRT1      48.35     51.65        100.0
Tata Motors_CV_ICV_SRT2      42.31     57.69        100.0
Tata Motors_CV_LCV_FTU1      47.58     52.42        100.0
Tata Motors_CV_LCV_FTU2      37.31     62.69        100.0
Tata Motors_CV_LCV_RTLC1     42.86     57.14        100.0
Tata Motors_CV_LCV_RTLC2     66.67     33.33        100.0
Tata Motors_CV_LCV_SRT1      44.12     55.88        100.0
Tata Motors_CV_LCV_SRT2      30.00     70.00        100.0
Tata Motors_CV_LCV_SRT3      75.00     25.00        100.0
Tata Motors_CV_MHCV_FTU1     58.03     41.97        100.0
Tata Motors_CV_MHCV_FTU2     40.53     59.47        100.0
Tata Motors_CV_MHCV_RTLC1    39.29     60.71        100.0
Tata Motors_CV

In [5]:
final_df = product_sampling.join(
    product_sampling_pct,
    how='left',
    lsuffix='_cnt',
    rsuffix='_pct'
)

print(final_df)



ApplicationStatus          Sampled_cnt  Screened_cnt  Grand Total_cnt  \
Product                                                                 
Tata Motors_CV_ICV_FTU1             67            51              118   
Tata Motors_CV_ICV_FTU2             59            71              130   
Tata Motors_CV_ICV_RTLC1             1             5                6   
Tata Motors_CV_ICV_SRT1             44            47               91   
Tata Motors_CV_ICV_SRT2             11            15               26   
Tata Motors_CV_LCV_FTU1             59            65              124   
Tata Motors_CV_LCV_FTU2             25            42               67   
Tata Motors_CV_LCV_RTLC1             3             4                7   
Tata Motors_CV_LCV_RTLC2            10             5               15   
Tata Motors_CV_LCV_SRT1             30            38               68   
Tata Motors_CV_LCV_SRT2              6            14               20   
Tata Motors_CV_LCV_SRT3              3             

In [6]:
df['tat_bucket'] = pd.cut(
    df['TAT In Minutes'],
    bins=[-1, 15, 30, float('inf')],
    labels=['<15 Min', '<30 Min', '>=30 Min']
)


In [7]:
tat_pivot = pd.pivot_table(
    df,
    index='Product',
    columns='tat_bucket',
    values='TAT In Minutes',
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name='Grand Total'
)

print(tat_pivot)


tat_bucket                 <15 Min  <30 Min  >=30 Min  Grand Total
Product                                                           
Tata Motors_CV_ICV_FTU1          3        4       111          118
Tata Motors_CV_ICV_FTU2          2        3       125          130
Tata Motors_CV_ICV_RTLC1         1        0         5            6
Tata Motors_CV_ICV_SRT1          0        2        89           91
Tata Motors_CV_ICV_SRT2          0        1        25           26
Tata Motors_CV_LCV_FTU1          1        6       117          124
Tata Motors_CV_LCV_FTU2          1        4        62           67
Tata Motors_CV_LCV_RTLC1         0        1         6            7
Tata Motors_CV_LCV_RTLC2         0        0        15           15
Tata Motors_CV_LCV_SRT1          1        2        65           68
Tata Motors_CV_LCV_SRT2          0        1        19           20
Tata Motors_CV_LCV_SRT3          0        0         4            4
Tata Motors_CV_MHCV_FTU1         0       20       366         

C:\Users\kpras\AppData\Local\Temp\ipykernel_7424\2481889891.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tat_pivot = pd.pivot_table(


In [8]:
tat_pct = tat_pivot.div(
    tat_pivot.drop(columns='Grand Total').sum(axis=1),
    axis=0
).mul(100).round(2)

print(tat_pct)


tat_bucket                 <15 Min  <30 Min  >=30 Min  Grand Total
Product                                                           
Tata Motors_CV_ICV_FTU1       2.54     3.39     94.07        100.0
Tata Motors_CV_ICV_FTU2       1.54     2.31     96.15        100.0
Tata Motors_CV_ICV_RTLC1     16.67     0.00     83.33        100.0
Tata Motors_CV_ICV_SRT1       0.00     2.20     97.80        100.0
Tata Motors_CV_ICV_SRT2       0.00     3.85     96.15        100.0
Tata Motors_CV_LCV_FTU1       0.81     4.84     94.35        100.0
Tata Motors_CV_LCV_FTU2       1.49     5.97     92.54        100.0
Tata Motors_CV_LCV_RTLC1      0.00    14.29     85.71        100.0
Tata Motors_CV_LCV_RTLC2      0.00     0.00    100.00        100.0
Tata Motors_CV_LCV_SRT1       1.47     2.94     95.59        100.0
Tata Motors_CV_LCV_SRT2       0.00     5.00     95.00        100.0
Tata Motors_CV_LCV_SRT3       0.00     0.00    100.00        100.0
Tata Motors_CV_MHCV_FTU1      0.00     5.18     94.82        1

In [9]:
tat_pct = tat_pct.add_suffix(' (%)')

final_tat = tat_pivot.join(tat_pct)
print(final_tat)


tat_bucket                 <15 Min  <30 Min  >=30 Min  Grand Total  \
Product                                                              
Tata Motors_CV_ICV_FTU1          3        4       111          118   
Tata Motors_CV_ICV_FTU2          2        3       125          130   
Tata Motors_CV_ICV_RTLC1         1        0         5            6   
Tata Motors_CV_ICV_SRT1          0        2        89           91   
Tata Motors_CV_ICV_SRT2          0        1        25           26   
Tata Motors_CV_LCV_FTU1          1        6       117          124   
Tata Motors_CV_LCV_FTU2          1        4        62           67   
Tata Motors_CV_LCV_RTLC1         0        1         6            7   
Tata Motors_CV_LCV_RTLC2         0        0        15           15   
Tata Motors_CV_LCV_SRT1          1        2        65           68   
Tata Motors_CV_LCV_SRT2          0        1        19           20   
Tata Motors_CV_LCV_SRT3          0        0         4            4   
Tata Motors_CV_MHCV_

In [10]:
final_df = (
    product_sampling.add_prefix('Status_')
        .join(product_sampling_pct.add_prefix('Status_').add_suffix(' (%)'))
        .join(tat_pivot.add_prefix('TAT_'))
        .join(tat_pct.add_prefix('TAT_').add_suffix(' (%)'))
)
print(final_df)

                           Status_Sampled  Status_Screened  \
Product                                                      
Tata Motors_CV_ICV_FTU1                67               51   
Tata Motors_CV_ICV_FTU2                59               71   
Tata Motors_CV_ICV_RTLC1                1                5   
Tata Motors_CV_ICV_SRT1                44               47   
Tata Motors_CV_ICV_SRT2                11               15   
Tata Motors_CV_LCV_FTU1                59               65   
Tata Motors_CV_LCV_FTU2                25               42   
Tata Motors_CV_LCV_RTLC1                3                4   
Tata Motors_CV_LCV_RTLC2               10                5   
Tata Motors_CV_LCV_SRT1                30               38   
Tata Motors_CV_LCV_SRT2                 6               14   
Tata Motors_CV_LCV_SRT3                 3                1   
Tata Motors_CV_MHCV_FTU1              224              162   
Tata Motors_CV_MHCV_FTU2               92              135   
Tata Mot

In [11]:
df = df.copy()
df["Process"] = df["Processed By"].str.strip().str.lower()
df["Process"] = df["Process"].apply(lambda x: "Auto" if "auto" in x else "Regular")

In [12]:
df = df.copy()
df["WorkingHours"] = df["Working Hours"].str.strip().str.lower()
df["WorkingHours"] = df["WorkingHours"].apply(lambda x: "Working Hours" if "9:30" in x else "Non Working Hours")

In [13]:
base_pivot = pd.pivot_table(
    df,
    index=["WorkingHours", "Process"],
    columns="tat_bucket",
    values="TAT In Minutes",
    aggfunc="count",
    fill_value=0
)
print(base_pivot)

tat_bucket                 <15 Min  <30 Min  >=30 Min
WorkingHours      Process                            
Non Working Hours Auto          52       21       562
                  Regular        5       15       736
Working Hours     Auto         352      283      1176
                  Regular       40      106      1916


C:\Users\kpras\AppData\Local\Temp\ipykernel_7424\1574690114.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  base_pivot = pd.pivot_table(


In [14]:
base_pivot["Grand Total"] = base_pivot.sum(axis=1)

In [15]:
subtotals = (base_pivot.groupby(level=0).sum())
subtotals.index = [f"{idx} Total" for idx in subtotals.index]


In [16]:
final_rows = []

for wh in base_pivot.index.get_level_values(0).unique():
    final_rows.append(base_pivot.loc[wh])
    final_rows.append(subtotals.loc[f"{wh} Total"].to_frame().T)

final_df2 = pd.concat(final_rows)



In [17]:
grand_total = pd.DataFrame(base_pivot.sum()).T
grand_total.index = ["Grand Total"]

final_df2 = pd.concat([final_df2, grand_total])


In [18]:
print(final_df2)

tat_bucket               <15 Min  <30 Min  >=30 Min  Grand Total
Auto                          52       21       562          635
Regular                        5       15       736          756
Non Working Hours Total       57       36      1298         1391
Auto                         352      283      1176         1811
Regular                       40      106      1916         2062
Working Hours Total          392      389      3092         3873
Grand Total                  449      425      4390         5264


In [19]:
pct_df = (
    final_df2[["<15 Min", "<30 Min", ">=30 Min"]]
    .div(final_df2["Grand Total"], axis=0)
    .mul(100)
    .round(0)
)
pct_df = pct_df.astype(int).astype(str) + "%"


In [20]:
pct_df.columns = [
    "<15 MIN %",
    "<30 MIN %",
    ">=30 MIN %"
]


In [21]:
final_df2 = pd.concat([final_df2, pct_df], axis=1)
print(final_df2)

                         <15 Min  <30 Min  >=30 Min  Grand Total <15 MIN %  \
Auto                          52       21       562          635        8%   
Regular                        5       15       736          756        1%   
Non Working Hours Total       57       36      1298         1391        4%   
Auto                         352      283      1176         1811       19%   
Regular                       40      106      1916         2062        2%   
Working Hours Total          392      389      3092         3873       10%   
Grand Total                  449      425      4390         5264        9%   

                        <30 MIN % >=30 MIN %  
Auto                           3%        89%  
Regular                        2%        97%  
Non Working Hours Total        3%        93%  
Auto                          16%        65%  
Regular                        5%        93%  
Working Hours Total           10%        80%  
Grand Total                    8%        83%  

In [25]:
df = df.copy()

df["Day wise Count of cases"] = df["Day"].apply(
    lambda x: "Holiday" if x.strip().lower() == "sunday" else "Regular"
)
day_type_counts = df["Day wise Count of cases"].value_counts()
print(day_type_counts)

Day wise Count of cases
Regular    5017
Holiday     247
Name: count, dtype: int64
